# Transformacion de los Datos para Modelado CLTV

Esta estapa es crucial para preparar el dataset para el modelado del CLTV con Redes Neuronales. Esta fase se realiza ya que no se pueden usar los datos tal cual como estan antes de esta etapa, se requiere transformar el registro de transacciones que esta en formato (filas por compras) a un formato de registros por cliente (una fila por cliente) estructurado en ventanas de tiempo.

In [21]:
import pandas as pd
from dateutil.relativedelta import relativedelta

df = pd.read_parquet('../data/processed/online_retail_II_cleaned.parquet')

fecha_inicio = df['invoicedate'].min().normalize()
fecha_fin = df['invoicedate'].max().normalize()

print(f"Rango de datos disponible: {fecha_inicio} a {fecha_fin}")

Rango de datos disponible: 2009-12-01 00:00:00 a 2010-11-30 00:00:00


In [22]:
df['totalamount'] = df['quantity'] * df['price']

# Verificación rápida
print("Estadísticas de totalamount:")
print(df['totalamount'].describe())

Estadísticas de totalamount:
count    386732.000000
mean         21.946918
std          77.236229
min           0.001000
25%           5.000000
50%          12.500000
75%          19.500000
max       15818.400391
Name: totalamount, dtype: float64


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386732 entries, 0 to 386731
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   invoice      386732 non-null  string        
 1   stockcode    386732 non-null  string        
 2   description  386732 non-null  string        
 3   quantity     386732 non-null  int16         
 4   invoicedate  386732 non-null  datetime64[ns]
 5   price        386732 non-null  float32       
 6   customer_id  386732 non-null  Int32         
 7   country      386732 non-null  string        
 8   totalamount  386732 non-null  float32       
dtypes: Int32(1), datetime64[ns](1), float32(2), int16(1), string(4)
memory usage: 20.3 MB


In [ ]:
ventanas = []

# Definimos las fechas de inicio de cada ventana de observación
# Solo podemos empezar en meses donde tengamos 9 meses por delante (3 obs + 6 target)
# Fechas viables: 2009-12-01, 2010-01-01, 2010-02-01, 2010-03-01
start_dates = [pd.Timestamp('2009-12-01') + relativedelta(months=i) for i in range(4)]

for i, start_date in enumerate(start_dates):
    # Definir fechas clave para esta ventana
    split_date = start_date + relativedelta(months=3)  # Fin de X, Inicio de y
    end_window_date = split_date + relativedelta(months=6) # Fin de y

    # Validar que no nos pasemos del final de los datos reales
    if end_window_date > fecha_fin + pd.Timedelta(days=5): # Margen de error pequeño
        print(f"Ventana {i+1} descartada: termina en {end_window_date}, data termina en {fecha_fin}")
        break

    print(f"Procesando Ventana {i+1}: Obs[{start_date.date()} a {split_date.date()}) -> Target[{split_date.date()} a {end_window_date.date()}]")

    # 1. Filtrar Datos para X (Observación - 3 meses)
    df_X = df[(df['invoicedate'] >= start_date) & (df['invoicedate'] < split_date)]

    # 2. Filtrar Datos para y (Target - 6 meses)
    df_y = df[(df['invoicedate'] >= split_date) & (df['invoicedate'] <= end_window_date)]

    # 3. Construir Features (X)
    X_window = df_X.groupby('customer_id').agg({
        'invoicedate': lambda x: (split_date - x.max()).days, # Recency relativa al corte
        'invoice': 'nunique',
        'totalamount': 'sum',
        'stockcode': 'nunique'
    }).reset_index()
    X_window.columns = ['customer_id', 'recency', 'frequency', 'monetary', 'product_variety']

    # Feature Contextual: Añadimos el mes de inicio para que el modelo sepa la estacionalidad
    X_window['Month_Start'] = start_date.month

    # 4. Construir Target (y)
    y_window = df_y.groupby('customer_id')['totalamount'].sum().reset_index()
    y_window.columns = ['customer_id', 'Target_CLTV_6m']

    # 5. Unir y Etiquetar
    window_data = pd.merge(X_window, y_window, on='customer_id', how='left')
    window_data['Target_CLTV_6m'] = window_data['Target_CLTV_6m'].fillna(0)

    # Agregar a la lista maestra
    ventanas.append(window_data)

# Concatenar todas las ventanas en un solo gran dataset
df_model_rolling = pd.concat(ventanas, ignore_index=True)

print(f"\nDataset Final Generado: {df_model_rolling.shape[0]} registros totales.")
print(f"Ejemplos promedio por cliente: {df_model_rolling.shape[0] / df_model_rolling['customer_id'].nunique():.2f}")
df_model_rolling.head()

Procesando Ventana 1: Obs[2009-12-01 a 2010-03-01) -> Target[2010-03-01 a 2010-09-01]
Procesando Ventana 2: Obs[2010-01-01 a 2010-04-01) -> Target[2010-04-01 a 2010-10-01]
Procesando Ventana 3: Obs[2010-02-01 a 2010-05-01) -> Target[2010-05-01 a 2010-11-01]
Procesando Ventana 4: Obs[2010-03-01 a 2010-06-01) -> Target[2010-06-01 a 2010-12-01]

Dataset Final Generado: 7509 registros totales.
Ejemplos promedio por cliente: 2.78


,customer_id,recency,frequency,monetary,product_variety,Month_Start,Target_CLTV_6m
0,12346,37,9,203.500000,2,12,169.360001
1,12358,82,1,1429.829956,17,12,268.100006
2,12359,74,2,838.890015,34,12,1173.140015
3,12360,6,1,118.000000,10,12,662.039978
4,12361,33,1,109.199997,7,12,98.150002


## Pipeline de Transformacion:

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler

In [ ]:
# ==============================================================================
# 1. LOG-TRANSFORMATION (Para corregir el sesgo de Pareto)
# ==============================================================================
# Aplicamos Logaritmo (log(x + 1)) directamente.
# Esto comprime los valores gigantes (outliers) y ayuda a la red neuronal a converger.
cols_to_log = ['Recency', 'Frequency', 'Monetary', 'ProductVariety', 'Target_CLTV_6m']

for col in cols_to_log:
    # Ya no hace falta filtrar negativos, aplicamos directo
    df_model_rolling[col] = np.log1p(df_model_rolling[col])

In [ ]:
# ==============================================================================
# 2. CODIFICACIÓN DE ESTACIONALIDAD (One-Hot Encoding)
# ==============================================================================
# Convertimos el mes de inicio de la ventana en columnas binarias
df_encoded = pd.get_dummies(df_model_rolling, columns=['Month_Start'], prefix='Start_Month', dtype=int)

In [ ]:
# ==============================================================================
# 3. DIVISIÓN TRAIN / TEST POR CLIENTE (Grouped Split)
# ==============================================================================
# Garantizamos que un cliente y todas sus ventanas temporales caigan en el mismo grupo
unique_customers = df_encoded['Customer ID'].unique()
n_customers = len(unique_customers)

# Semilla para reproducibilidad
np.random.seed(42)
shuffled_ids = np.random.permutation(unique_customers)

# 80% Entrenamiento / 20% Prueba
split_idx = int(n_customers * 0.8)
train_ids = shuffled_ids[:split_idx]
test_ids = shuffled_ids[split_idx:]

# Filtramos el DataFrame maestro
train_df = df_encoded[df_encoded['Customer ID'].isin(train_ids)]
test_df = df_encoded[df_encoded['Customer ID'].isin(test_ids)]

print(f"Dataset de Entrenamiento: {len(train_ids)} clientes | {len(train_df)} ventanas")
print(f"Dataset de Prueba:        {len(test_ids)} clientes | {len(test_df)} ventanas")

In [ ]:
# ==============================================================================
# 4. ESCALADO ESTÁNDAR (Z-Score)
# ==============================================================================
features_col = [c for c in df_encoded.columns if c not in ['Customer ID', 'Target_CLTV_6m']]
target_col = 'Target_CLTV_6m'

X_train = train_df[features_col].values
y_train = train_df[target_col].values

X_test = test_df[features_col].values
y_test = test_df[target_col].values

# El scaler aprende SOLO del set de entrenamiento para evitar "Data Leakage"
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test) # Solo transforma el test, no aprende de él

print(f"\nDimensiones listas para la Red Neuronal:")
print(f"Inputs (X): {X_train_scaled.shape}")